## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,41.11,87,75,2.30,broken clouds
1,1,Hithadhoo,MV,-0.6000,73.0833,78.33,85,100,12.62,overcast clouds
2,2,Bluff,NZ,-46.6000,168.3333,53.80,70,99,17.65,overcast clouds
3,3,Krasnovishersk,RU,60.4078,57.0820,40.84,80,5,2.44,clear sky
4,4,Cap-Aux-Meules,CA,47.3821,-61.8597,57.22,100,61,5.75,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input 
#statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hithadhoo,MV,-0.6000,73.0833,78.33,85,100,12.62,overcast clouds
5,5,Kutum,SD,14.2000,24.6667,75.43,24,98,8.03,overcast clouds
19,19,San Patricio,US,28.0170,-97.5169,77.83,51,0,7.92,clear sky
20,20,Vaini,TO,-21.2000,-175.2000,78.96,83,75,10.36,light rain
21,21,Nioro,GM,13.3500,-15.7500,76.01,95,100,1.54,overcast clouds
23,23,Atuona,PF,-9.8000,-139.0333,78.21,73,30,16.64,scattered clouds
25,25,Ixtapa,MX,20.7000,-105.2000,75.22,94,90,11.50,overcast clouds
31,31,Kapaa,US,22.0752,-159.3190,82.38,73,100,1.01,light rain
32,32,Bac Lieu,VN,9.2850,105.7244,85.87,66,100,9.93,overcast clouds
35,35,Sao Filipe,CV,14.8961,-24.4956,78.28,77,100,4.72,overcast clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 4b. Drop any empty rows and create a new DataFrame that 
#doesn’t have empty rows.Drop the NaNs.
#preferred_cities_df.dropna()

In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.nan
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hithadhoo,MV,78.33,overcast clouds,-0.6000,73.0833,NaN
5,Kutum,SD,75.43,overcast clouds,14.2000,24.6667,NaN
19,San Patricio,US,77.83,clear sky,28.0170,-97.5169,NaN
20,Vaini,TO,78.96,light rain,-21.2000,-175.2000,NaN
21,Nioro,GM,76.01,overcast clouds,13.3500,-15.7500,NaN
23,Atuona,PF,78.21,scattered clouds,-9.8000,-139.0333,NaN
25,Ixtapa,MX,75.22,overcast clouds,20.7000,-105.2000,NaN
31,Kapaa,US,82.38,light rain,22.0752,-159.3190,NaN
32,Bac Lieu,VN,85.87,overcast clouds,9.2850,105.7244,NaN
35,Sao Filipe,CV,78.28,overcast clouds,14.8961,-24.4956,NaN


In [11]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
 
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to 
    #get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the
    #search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the 
    #name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
        # Grab the first hotel from the results and store the name.
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [13]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hithadhoo,MV,78.33,overcast clouds,-0.6000,73.0833,Scoop Guest House
5,Kutum,SD,75.43,overcast clouds,14.2000,24.6667,Dibbo areaمنطقة دبو
19,San Patricio,US,77.83,clear sky,28.0170,-97.5169,"Motel 6 Sinton, TX"
20,Vaini,TO,78.96,light rain,-21.2000,-175.2000,Keleti Beach Resort
23,Atuona,PF,78.21,scattered clouds,-9.8000,-139.0333,Villa Enata
...,...,...,...,...,...,...,...
704,Encheng,CN,86.02,overcast clouds,22.1839,112.3026,Pinghu Hostel
711,Port-Gentil,GA,76.98,overcast clouds,-0.7193,8.7815,Fengshui Residence
713,Umm Lajj,SA,82.51,clear sky,25.0213,37.2685,Moon light Furnished Units
714,Samana,DO,76.93,overcast clouds,19.2056,-69.3369,The Bannister - Hotel & Yacht Club


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and marker
#for each city.
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Call the figure to plot the data.
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
#fig = (heat_layer, marker_layer)

In [34]:
# Save the figure.
#fig.savefig("Vacation_Search/WeatherPy_vacation_map.png")